# report

In [6]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
import copy
import holidays
nyse_holidays = holidays.financial_holidays('NYSE')
import datetime
import time
from contract_select import contract_select
from hedge_calc import hedge_calc
from greek_pnl_calc import greek_calc, pnl_daily_calc, get_next_trading_day
from position_adj_cost import position_adj_cost, initial_position_cost

# Set the first trading date

In [2]:
moneyness = 'out'
main_type = 'P'
hedge_type = 'P'
main_minDistance = 0.08
hedge_minDistance = 0.06
starting_date = '2018-12-10'

main_original = contract_select(select_type='main', date= starting_date, max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk=1.0, minDistance = main_minDistance, minDTE=15, moneyness=moneyness, num = 2)
hedge_original = contract_select(select_type='hedge', date= starting_date, max_spread_pct = 0.05, spy_spx='spx', contract_type = hedge_type, minAsk=0.2, minDistance = hedge_minDistance, minDTE=15, moneyness=moneyness, num = 3)
main_original.update(hedge_original)

initial_position = hedge_calc(main_original)
initial_spread_cost = initial_position_cost(initial_position)

# Wind up position every friday and day before any holiday

In [4]:
greek_pnl_daily = pd.DataFrame()

position = copy.deepcopy(initial_position)
position['date'] = starting_date

greek_pnl_daily = greek_pnl_daily.append(greek_calc(position))
greek_pnl_daily['daily_pnl'] = 0
greek_pnl_daily['adjust_cost'] = initial_spread_cost['adjust_cost']
daily = []


for i in range(1):
    if len(position['main']) == 0 and len(position['hedge']) == 0:
        position['date'] =  get_next_trading_day(position['date'], position['main_type'])
        position['daily_rtn'] = 0
    else:
        position = pnl_daily_calc(position)
        greek_pnl_daily['daily_pnl'][-1] =  position['daily_rtn']

    date = position['date']
    #get the date object of next day (not next trading day)
    date_1 = datetime.date.fromisoformat(date) + datetime.timedelta(days=1)
    
    temp = copy.deepcopy(position)

    # determine whther today is friday or day before any holiday
    if datetime.date.weekday(datetime.date.fromisoformat(date)) == 4 or (date_1 in nyse_holidays):
        temp['main'] = temp['main'].iloc[0:0]
        temp['hedge'] = temp['hedge'].iloc[0:0]
        temp['emini'] = temp['emini'].iloc[0:0]
        greek_1 = pd.DataFrame(index = [date], data = {'DELTA':0, 'GAMMA':0, 'VEGA':0, 'THETA':0, 'daily_pnl':0, 'adjust_cost':0})
        greek_pnl_daily = greek_pnl_daily.append(greek_1)
        position=copy.deepcopy(temp)
        
    else:    
        # drop/wind up certain main contracts that meet some criteria
        if len(temp['main']) != 0:
            if main_type == 'P': 
                #drop ITM contracts
                temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] > temp['main']['[STRIKE]']]
                #minimal ask needs to be above 0.5
                # temp['main'] = temp['main'][temp['main']['[P_ASK]'] > 0.5]
            if main_type == 'C': 
                #drop ITM contracts
                temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] < temp['main']['[STRIKE]']]
                #minimal ask needs to be above 0.5
                # temp['main'] = temp['main'][temp['main']['[C_ASK]'] > 0.5]
            #minimum DTE need to be at least 5 days
            temp['main'] = temp['main'][temp['main']['[DTE]'] > 5]
            #minimum distance need to be half the main_minDistance
            temp['main'] = temp['main'][temp['main']['[STRIKE_DISTANCE_PCT]'] >= main_minDistance/2]
        #select new contracts if num of contracts is less than 2
        if len(temp['main']) < 2:
            new_contract = contract_select(select_type='main', date = date, max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk = 1.0, minDistance = main_minDistance, minDTE = 15, moneyness=moneyness, num=2)
            temp['main'] = temp['main'].append(new_contract['main'])
            temp['main'] = temp['main'][temp['main'].columns[:11]].drop_duplicates()
            temp['main'] = temp['main'].iloc[:2]
            
        # drop/wind up certain hedge contracts that meet some criteria
        if len(temp['hedge']) != 0:
            if hedge_type == 'P': 
                #drop ITM contracts
                temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] > temp['hedge']['[STRIKE]']]
            if hedge_type == 'C': 
                #drop ITM contracts
                temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] < temp['hedge']['[STRIKE]']]
            #minimum DTE need to be at least 5 days
            temp['hedge'] = temp['hedge'][temp['hedge']['[DTE]'] > 5]
            #minimum distance need to be half the main_minDistance
            temp['hedge'] = temp['hedge'][temp['hedge']['[STRIKE_DISTANCE_PCT]'] >= hedge_minDistance/2]
        #select new contracts if num of contracts is less than 3
        if len(temp['hedge']) < 3:
                new_contract = contract_select(select_type = 'hedge', date = date, max_spread_pct = 1, spy_spx = 'spx', contract_type = hedge_type, minAsk = 0.2, minDistance = hedge_minDistance, minDTE = 15, moneyness=moneyness, num = 3)
                temp['hedge'] = temp['hedge'].append(new_contract['hedge'])
                temp['hedge'] = temp['hedge'][temp['hedge'].columns[:11]].drop_duplicates()
                temp['hedge'] = temp['hedge'].iloc[:3]
        #if no contracts are available to select for either main and hedge, set the main and hedge to null
        if len(temp['main']) == 0 or len(temp['hedge']) == 0:
            temp['main'] = temp['main'].iloc[0:0]
            temp['hedge'] = temp['hedge'].iloc[0:0]
            temp['emini'] = temp['emini'].iloc[0:0]
            greek_1 = pd.DataFrame(index = [date], data = {'DELTA':0, 'GAMMA':0, 'VEGA':0, 'THETA':0, 'daily_pnl':0, 'adjust_cost':0})
            greek_pnl_daily = greek_pnl_daily.append(greek_1)
            position=copy.deepcopy(temp)
        else:        
            temp = hedge_calc(temp)
            greek_pnl_daily = greek_pnl_daily.append(greek_calc(temp))
    
    if len(position['main']) == 0 and len(position['hedge']) == 0 and len(temp['main']) == 0 and len(temp['hedge']) == 0:
        greek_pnl_daily['adjust_cost'][-1] = 0
    elif len(position['main']) == 0 and len(position['hedge']) == 0:
        greek_pnl_daily['adjust_cost'][-1] = initial_position_cost(temp)['adjust_cost']
    elif len(temp['main']) == 0 and len(temp['hedge']) == 0:
        greek_pnl_daily['adjust_cost'][-1] = 0
    else:
        greek_pnl_daily['adjust_cost'][-1] = position_adj_cost(position, temp)['adjust_cost']
    
    position=copy.deepcopy(temp)
    daily.append(position)
    
greek_pnl_daily['daily_pnl'] = greek_pnl_daily['daily_pnl'].shift(1)
greek_pnl_daily['daily_pnl'][0] = 0

In [ ]:
greek_pnl_daily.to_clipboard()

In [5]:
greek_pnl_daily

,DELTA,GAMMA,VEGA,THETA,daily_pnl,adjust_cost
2018-12-10,0.0,-0.000657,-0.636530,0.499719,0.000000,-0.005528
2018-12-11,0.0,-0.000661,-0.572170,0.463749,0.006287,-0.000027
2018-12-12,0.0,-0.000563,-0.439580,0.367898,0.004261,-0.000123
2018-12-13,0.0,-0.000498,-0.367274,0.323286,-0.000015,-0.000448
2018-12-14,0.0,0.000000,0.000000,0.000000,0.014238,0.000000
...,...,...,...,...,...,...
2019-12-09,0.0,-0.000490,-0.475887,0.388403,0.000000,-0.003903
2019-12-10,0.0,-0.000441,-0.443672,0.385834,0.002965,-0.000124
2019-12-11,0.0,-0.000448,-0.343541,0.309338,0.004764,-0.000073
2019-12-12,0.0,-0.000255,-0.233509,0.237366,-0.002311,-0.000180


# Ignore any holiday and friday

In [ ]:
greek_pnl_daily = pd.DataFrame()

position = copy.deepcopy(initial_position)
position['date'] = starting_date

greek_pnl_daily = greek_pnl_daily.append(greek_calc(position))
greek_pnl_daily['daily_pnl'] = 0
greek_pnl_daily['adjust_cost'] = initial_spread_cost['adjust_cost']
daily = []

for i in range(500):
    position = pnl_daily_calc(position)
    greek_pnl_daily['daily_pnl'][-1] =  position['daily_rtn']
    date = position['date']
    daily.append(position)
           
    temp = copy.deepcopy(position)
    # drop/wind up certain main contracts that meet some criteria
    if len(temp['main']) != 0:
        if main_type == 'P': 
            #drop ITM contracts
            temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] > temp['main']['[STRIKE]']]
            #minimal ask needs to be above 0.5
            # temp['main'] = temp['main'][temp['main']['[P_ASK]'] > 0.5]
        if main_type == 'C': 
            #drop ITM contracts
            temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] < temp['main']['[STRIKE]']]
            #minimal ask needs to be above 0.5
            # temp['main'] = temp['main'][temp['main']['[C_ASK]'] > 0.5]
        #minimum DTE need to be at least 5 days
        temp['main'] = temp['main'][temp['main']['[DTE]'] > 5]
        #minimum distance need to be half the main_minDistance
        temp['main'] = temp['main'][temp['main']['[STRIKE_DISTANCE_PCT]'] >= main_minDistance/2]
    #select new contracts if num of contracts is less than 2
    if len(temp['main']) < 2:
        new_contract = contract_select(select_type='main', date = date, max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk = 1.0, minDistance = main_minDistance, minDTE = 15, moneyness=moneyness, num=2-len(temp['main']))
        temp['main'] = temp['main'].append(new_contract['main'])

    # drop/wind up certain hedge contracts that meet some criteria
    if len(temp['hedge']) != 0:
        if hedge_type == 'P': 
            #drop ITM contracts
            temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] > temp['hedge']['[STRIKE]']]
        if hedge_type == 'C': 
            #drop ITM contracts
            temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] < temp['hedge']['[STRIKE]']]
        #minimum DTE need to be at least 5 days
        temp['hedge'] = temp['hedge'][temp['hedge']['[DTE]'] > 5]
        #minimum distance need to be half the main_minDistance
        temp['hedge'] = temp['hedge'][temp['hedge']['[STRIKE_DISTANCE_PCT]'] >= hedge_minDistance/2]
    #select new contracts if num of contracts is less than 3
    if len(temp['hedge']) < 3:
            new_contract = contract_select(select_type = 'hedge', date = date, max_spread_pct = 1, spy_spx = 'spx', contract_type = hedge_type, minAsk = 0.2, minDistance = hedge_minDistance, minDTE = 15, moneyness=moneyness, num = 3 - len(temp['hedge']))
            temp['hedge'] = temp['hedge'].append(new_contract['hedge'])
    
    temp = hedge_calc(temp)
    greek_pnl_daily = greek_pnl_daily.append(greek_calc(position))
    greek_pnl_daily['adjust_cost'][-1] = position_adj_cost(position, temp)['adjust_cost']
    position=copy.deepcopy(temp)
    
greek_pnl_daily['daily_pnl'] = greek_pnl_daily['daily_pnl'].shift(1)
greek_pnl_daily['daily_pnl'][0] = 0

In [ ]:
greek_pnl_daily.to_clipboard()

In [ ]:
greek_pnl_daily

# Wind up position every friday

In [ ]:
greek_pnl_daily = pd.DataFrame()

position = copy.deepcopy(initial_position)
position['date'] = starting_date

greek_pnl_daily = greek_pnl_daily.append(greek_calc(position))
greek_pnl_daily['daily_pnl'] = 0
greek_pnl_daily['adjust_cost'] = initial_spread_cost['adjust_cost']
daily = []


for i in range(500):
    if len(position['main']) == 0 and len(position['hedge']) == 0:
        position['date'] =  get_next_trading_day(position['date'], position['main_type'])
        position['daily_rtn'] = 0
    else:
        position = pnl_daily_calc(position)
        greek_pnl_daily['daily_pnl'][-1] =  position['daily_rtn']

    date = position['date']
    #get the date object of next day (not next trading day)
    date_1 = datetime.date.fromisoformat(date) + datetime.timedelta(days=1)
    
    temp = copy.deepcopy(position)

    # determine whther today is friday
    if datetime.date.weekday(datetime.date.fromisoformat(date)) == 4:
        temp['main'] = temp['main'].iloc[0:0]
        temp['hedge'] = temp['hedge'].iloc[0:0]
        temp['emini'] = temp['emini'].iloc[0:0]
        greek_1 = pd.DataFrame(index = [date], data = {'DELTA':0, 'GAMMA':0, 'VEGA':0, 'THETA':0, 'daily_pnl':0, 'adjust_cost':0})
        greek_pnl_daily = greek_pnl_daily.append(greek_1)
        position=copy.deepcopy(temp)
        
    else:    
        # drop/wind up certain main contracts that meet some criteria
        if len(temp['main']) != 0:
            if main_type == 'P': 
                #drop ITM contracts
                temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] > temp['main']['[STRIKE]']]
                #minimal ask needs to be above 0.5
                # temp['main'] = temp['main'][temp['main']['[P_ASK]'] > 0.5]
            if main_type == 'C': 
                #drop ITM contracts
                temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] < temp['main']['[STRIKE]']]
                #minimal ask needs to be above 0.5
                # temp['main'] = temp['main'][temp['main']['[C_ASK]'] > 0.5]
            #minimum DTE need to be at least 5 days
            temp['main'] = temp['main'][temp['main']['[DTE]'] > 5]
            #minimum distance need to be half the main_minDistance
            temp['main'] = temp['main'][temp['main']['[STRIKE_DISTANCE_PCT]'] >= main_minDistance/2]
        #select new contracts if num of contracts is less than 2
        if len(temp['main']) < 2:
            new_contract = contract_select(select_type='main', date = date, max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk = 1.0, minDistance = main_minDistance, minDTE = 15, moneyness=moneyness, num=2)
            temp['main'] = temp['main'].append(new_contract['main'])
            temp['main'] = temp['main'][temp['main'].columns[:11]].drop_duplicates()
            temp['main'] = temp['main'].iloc[:2]
            
        # drop/wind up certain hedge contracts that meet some criteria
        if len(temp['hedge']) != 0:
            if hedge_type == 'P': 
                #drop ITM contracts
                temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] > temp['hedge']['[STRIKE]']]
            if hedge_type == 'C': 
                #drop ITM contracts
                temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] < temp['hedge']['[STRIKE]']]
            #minimum DTE need to be at least 5 days
            temp['hedge'] = temp['hedge'][temp['hedge']['[DTE]'] > 5]
            #minimum distance need to be half the main_minDistance
            temp['hedge'] = temp['hedge'][temp['hedge']['[STRIKE_DISTANCE_PCT]'] >= hedge_minDistance/2]
        #select new contracts if num of contracts is less than 3
        if len(temp['hedge']) < 3:
                new_contract = contract_select(select_type = 'hedge', date = date, max_spread_pct = 1, spy_spx = 'spx', contract_type = hedge_type, minAsk = 0.2, minDistance = hedge_minDistance, minDTE = 15, moneyness=moneyness, num = 3)
                temp['hedge'] = temp['hedge'].append(new_contract['hedge'])
                temp['hedge'] = temp['hedge'][temp['hedge'].columns[:11]].drop_duplicates()
                temp['hedge'] = temp['hedge'].iloc[:3]
        #if no contracts are available to select for either main and hedge, set the main and hedge to null
        if len(temp['main']) == 0 or len(temp['hedge']) == 0:
            temp['main'] = temp['main'].iloc[0:0]
            temp['hedge'] = temp['hedge'].iloc[0:0]
            temp['emini'] = temp['emini'].iloc[0:0]
            greek_1 = pd.DataFrame(index = [date], data = {'DELTA':0, 'GAMMA':0, 'VEGA':0, 'THETA':0, 'daily_pnl':0, 'adjust_cost':0})
            greek_pnl_daily = greek_pnl_daily.append(greek_1)
            position=copy.deepcopy(temp)
        else:        
            temp = hedge_calc(temp)
            greek_pnl_daily = greek_pnl_daily.append(greek_calc(temp))
    
    if len(position['main']) == 0 and len(position['hedge']) == 0 and len(temp['main']) == 0 and len(temp['hedge']) == 0:
        greek_pnl_daily['adjust_cost'][-1] = 0
    elif len(position['main']) == 0 and len(position['hedge']) == 0:
        greek_pnl_daily['adjust_cost'][-1] = initial_position_cost(temp)['adjust_cost']
    elif len(temp['main']) == 0 and len(temp['hedge']) == 0:
        greek_pnl_daily['adjust_cost'][-1] = 0
    else:
        greek_pnl_daily['adjust_cost'][-1] = position_adj_cost(position, temp)['adjust_cost']
    
    position=copy.deepcopy(temp)
    daily.append(position)
    
greek_pnl_daily['daily_pnl'] = greek_pnl_daily['daily_pnl'].shift(1)
greek_pnl_daily['daily_pnl'][0] = 0

In [ ]:
greek_pnl_daily.to_clipboard()

In [ ]:
greek_pnl_daily

# Wind up position day before any holiday

In [ ]:
greek_pnl_daily = pd.DataFrame()

position = copy.deepcopy(initial_position)
position['date'] = starting_date

greek_pnl_daily = greek_pnl_daily.append(greek_calc(position))
greek_pnl_daily['daily_pnl'] = 0
greek_pnl_daily['adjust_cost'] = initial_spread_cost['adjust_cost']
daily = []


for i in range(500):
    if len(position['main']) == 0 and len(position['hedge']) == 0:
        position['date'] =  get_next_trading_day(position['date'], position['main_type'])
        position['daily_rtn'] = 0
    else:
        position = pnl_daily_calc(position)
        greek_pnl_daily['daily_pnl'][-1] =  position['daily_rtn']

    date = position['date']
    #get the date object of next day (not next trading day)
    date_1 = datetime.date.fromisoformat(date) + datetime.timedelta(days=1)
    
    temp = copy.deepcopy(position)

    # determine whether next day is holiday
    if date_1 in nyse_holidays:
        temp['main'] = temp['main'].iloc[0:0]
        temp['hedge'] = temp['hedge'].iloc[0:0]
        temp['emini'] = temp['emini'].iloc[0:0]
        greek_1 = pd.DataFrame(index = [date], data = {'DELTA':0, 'GAMMA':0, 'VEGA':0, 'THETA':0, 'daily_pnl':0, 'adjust_cost':0})
        greek_pnl_daily = greek_pnl_daily.append(greek_1)
        position=copy.deepcopy(temp)
        
    else:    
        # drop/wind up certain main contracts that meet some criteria
        if len(temp['main']) != 0:
            if main_type == 'P': 
                #drop ITM contracts
                temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] > temp['main']['[STRIKE]']]
                #minimal ask needs to be above 0.5
                # temp['main'] = temp['main'][temp['main']['[P_ASK]'] > 0.5]
            if main_type == 'C': 
                #drop ITM contracts
                temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] < temp['main']['[STRIKE]']]
                #minimal ask needs to be above 0.5
                # temp['main'] = temp['main'][temp['main']['[C_ASK]'] > 0.5]
            #minimum DTE need to be at least 5 days
            temp['main'] = temp['main'][temp['main']['[DTE]'] > 5]
            #minimum distance need to be half the main_minDistance
            temp['main'] = temp['main'][temp['main']['[STRIKE_DISTANCE_PCT]'] >= main_minDistance/2]
        #select new contracts if num of contracts is less than 2
        if len(temp['main']) < 2:
            new_contract = contract_select(select_type='main', date = date, max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk = 1.0, minDistance = main_minDistance, minDTE = 15, moneyness=moneyness, num=2)
            temp['main'] = temp['main'].append(new_contract['main'])
            temp['main'] = temp['main'][temp['main'].columns[:11]].drop_duplicates()
            temp['main'] = temp['main'].iloc[:2]
            
        # drop/wind up certain hedge contracts that meet some criteria
        if len(temp['hedge']) != 0:
            if hedge_type == 'P': 
                #drop ITM contracts
                temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] > temp['hedge']['[STRIKE]']]
            if hedge_type == 'C': 
                #drop ITM contracts
                temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] < temp['hedge']['[STRIKE]']]
            #minimum DTE need to be at least 5 days
            temp['hedge'] = temp['hedge'][temp['hedge']['[DTE]'] > 5]
            #minimum distance need to be half the main_minDistance
            temp['hedge'] = temp['hedge'][temp['hedge']['[STRIKE_DISTANCE_PCT]'] >= hedge_minDistance/2]
        #select new contracts if num of contracts is less than 3
        if len(temp['hedge']) < 3:
                new_contract = contract_select(select_type = 'hedge', date = date, max_spread_pct = 1, spy_spx = 'spx', contract_type = hedge_type, minAsk = 0.2, minDistance = hedge_minDistance, minDTE = 15, moneyness=moneyness, num = 3)
                temp['hedge'] = temp['hedge'].append(new_contract['hedge'])
                temp['hedge'] = temp['hedge'][temp['hedge'].columns[:11]].drop_duplicates()
                temp['hedge'] = temp['hedge'].iloc[:3]
        #if no contracts are available to select for either main and hedge, set the main and hedge to null
        if len(temp['main']) == 0 or len(temp['hedge']) == 0:
            temp['main'] = temp['main'].iloc[0:0]
            temp['hedge'] = temp['hedge'].iloc[0:0]
            temp['emini'] = temp['emini'].iloc[0:0]
            greek_1 = pd.DataFrame(index = [date], data = {'DELTA':0, 'GAMMA':0, 'VEGA':0, 'THETA':0, 'daily_pnl':0, 'adjust_cost':0})
            greek_pnl_daily = greek_pnl_daily.append(greek_1)
            position=copy.deepcopy(temp)
        else:        
            temp = hedge_calc(temp)
            greek_pnl_daily = greek_pnl_daily.append(greek_calc(temp))
    
    if len(position['main']) == 0 and len(position['hedge']) == 0 and len(temp['main']) == 0 and len(temp['hedge']) == 0:
        greek_pnl_daily['adjust_cost'][-1] = 0
    elif len(position['main']) == 0 and len(position['hedge']) == 0:
        greek_pnl_daily['adjust_cost'][-1] = initial_position_cost(temp)['adjust_cost']
    elif len(temp['main']) == 0 and len(temp['hedge']) == 0:
        greek_pnl_daily['adjust_cost'][-1] = 0
    else:
        greek_pnl_daily['adjust_cost'][-1] = position_adj_cost(position, temp)['adjust_cost']
    
    position=copy.deepcopy(temp)
    daily.append(position)
    
greek_pnl_daily['daily_pnl'] = greek_pnl_daily['daily_pnl'].shift(1)
greek_pnl_daily['daily_pnl'][0] = 0

In [ ]:
greek_pnl_daily.to_clipboard()

In [ ]:
greek_pnl_daily

In [ ]:
import pickle
with open('dat_1', 'wb') as fp:
    pickle.dump(dat_1, fp)